# Финальный отчёт

Спустя столько времени построения моделей разными способами, захотелось вернуться к любимой теме потроения карт с предсказаниями.
Так же в этом отчёте есть интерактивные графики, показывающие разницу между между реальным и предсказанным числом поездок.
Данный функционал обновляется вместе с картой (по времени и региону).
Далее можно посмотреть 
В заключение представлен скрипт, который может экспортировать графики в html (plotly).
К сожалению с виджетами сделать этого не удалось.
Проект представлен AS IS,при установке соответсвующих модулей, описанных ниже, ноутбук должен отработать.
Примеры графиков представлены в final_report.html

### Необходимы модули (python 3.6):
- карты ipyleaflet
- интерактивные виджеты графиков plotly
- виджеты ipywidgets

In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from datetime import timedelta

# интерактивный график
import plotly

# отображение карт
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

from traitlets import link

# виджеты
from ipywidgets import interact, interactive, fixed
from ipywidgets import widgets
from IPython.display import display, clear_output, HTML

# для регионов и их раскраски
import geojson, json
import branca.colormap as cm

%matplotlib inline

In [64]:
plotly.offline.init_notebook_mode(connected=True)

Границы Нью-Йорка

In [65]:
ny_west = -74.25559
ny_east = -73.70001
ny_north = 40.91553
ny_south = 40.49612

In [66]:
current_values = {}
true_values = []
pred_values = []
selected_region = 1075

Загружаем данные о регионах, о фактических поездках и предсказания построенные с помощью регрессоров.

In [67]:
# geojson данных прогнозируемых регионов
with open("processed_data/regions.json", "r") as fid:
    geo_data = json.load(fid)
    geo_data_pred = geo_data.copy()    
g_true = GeoJSON(data=geo_data)
g_pred = GeoJSON(data=geo_data_pred)

In [68]:
# данные о поездках из регионов
data = pd.read_csv("processed_data/all_trip_data.csv", index_col='time')
data.index = pd.to_datetime(data.index)

In [69]:
# список регионов
regions = data.columns.astype(int).tolist()

In [70]:
# центры регионов
regions_pos = pd.read_csv("raw_data/regions.csv", sep=";", index_col=['region'])
regions_pos['center_lat'] = (regions_pos['north'] + regions_pos['south'])/2
regions_pos['center_lon'] = (regions_pos['east'] + regions_pos['west'])/2

In [71]:
# Данные за июнь
df = data['2016-05-31 23:00:00':]

In [72]:
# загружаем предсказания
data_submit = pd.read_csv("submitions/w6_best.csv")

id_submit = data_submit.id.apply(lambda x: x.split("_"))
data_pred = pd.DataFrame({'y': data_submit.y,
                          'region': [x[0] for x in id_submit],
                          'date': [x[1] for x in id_submit],
                          'hour': [x[2] for x in id_submit],
                          'hour_pred': [x[3] for x in id_submit]})

In [73]:
data_pred = data_pred.groupby(['region','date','hour','hour_pred']).y.mean().reset_index()

In [74]:
# Аггрегируем данные по часам, регионам, датам
hourly_data = data_pred.groupby(by=['region', 'date', 'hour'])

region_data = data_pred.groupby(by=['region', 'hour_pred'])

date_data = data_pred.groupby(by=['date', 'hour', 'hour_pred'])

In [75]:
# Функция для обработки пользовательских событий
def dateChanged(change):
    pass

def printRegion(event=None, id=None, properties=None):
    label_current.value = u"Регион: " + properties['region']
    
def printRegionPred(event=None, id=None, properties=None):
    global selected_region
    region = properties['region']
    label_region_clicked.value = u"Регион: " + region
    label_current.value = u"Текущее значение: " + str(current_values.loc[region])
    label_true.value = u"Через " + str(prediction_hour_slider.value) + u" часа(ов): " + str(true_values.loc[region])
    label_pred.value = u"Предсказание: " + str(pred_values.loc[region])
    
    show_graph(int(region), prediction_hour_slider.value)
    selected_region = region
    
def on_pred_hour_changed(change):
    global selected_region
    if selected_region is not None:
        show_graph(int(selected_region), prediction_hour_slider.value)

def update_colors(geo_json_data, data_values, color_function):
    for feature, value in zip(geo_json_data['features'], data_values):
        feature['properties']['style'] = {'color':'black', 'weight': 1, 'fillColor':color_function(value), 'fillOpacity':0.4}

def region_selected(change):
    region = int(change['new'])
    ny_map_real.center = (regions_pos.loc[region, 'center_lat'], regions_pos.loc[region, 'center_lon'])
    ny_map_real.zoom = 14
        

In [100]:
def setLayout(change):
    global g_true
    global g_pred
    global current_values, true_values, pred_values
    
    # Checking date
    if date_selector_map.value.to_pydatetime() == datetime(year=2016, month=5, day=31) and hour_selector_map.value != 23:
        hour_selector_map.set_trait('value', 23)
        return
    if date_selector_map.value.to_pydatetime() == datetime(year=2016, month=6, day=30) and hour_selector_map.value > 17:
        hour_selector_map.set_trait('value', 17)
        return
    # удаляем старый слой, вставляя новый
    try:
        ny_map_real.remove_layer(g_true)
        ny_map_pred.remove_layer(g_pred)
    except:
        pass
    finally:
        
        if selected_region is not None:
            show_graph(int(selected_region), prediction_hour_slider.value)
        hour_horizon = prediction_hour_slider.value
        td = date_selector_map.value
        current_hour = hour_selector_map.value
        
        current_date = pd.DatetimeIndex([td.to_pydatetime().replace(hour=current_hour)], freq='H')[0]
        
        # 
        current_values = df.loc[current_date]
        true_values = df.loc[current_date+hour_horizon]
        pred_values = date_data.get_group((td.strftime('%Y-%m-%d'), str(current_hour), str(hour_horizon)))
        pred_values = pred_values.set_index('region')
        pred_values = pred_values.y
        
        # цвета регионов
        c_v_max, c_v_min = current_values.max(), current_values.min()
        t_v_max, t_v_min = true_values.max(), true_values.min()
        p_v_max, p_v_min = pred_values.max(), pred_values.min()
        vmax = max(c_v_max, t_v_max, p_v_max)
        vmin = max(c_v_min, t_v_min, p_v_min)
        width = vmax - vmin
        c_func = cm.LinearColormap(['green', 'yellow','red'],
                                   vmax = vmax, 
                                   vmin = vmin,
                                   index=[vmin, 0.5*width, 1.0*vmax])
        
        # geojson data
        update_colors(geo_data, current_values.values, c_func)
        update_colors(geo_data_pred, pred_values.values, c_func)
        
        g_true = GeoJSON(data=geo_data, hover_style={'fillColor': 'blue'})
        g_pred = GeoJSON(data=geo_data_pred, hover_style={'fillColor': 'blue'})
        
        # взаимосвязь с регионами
        g_true.on_hover(printRegionPred)
        g_pred.on_hover(printRegionPred)
#         g_true.on_click(printRegionPred)
#         g_pred.on_click(printRegionPred)
        
        ny_map_real.add_layer(g_true)
        ny_map_pred.add_layer(g_pred)

In [101]:
# отображение интерактивного графика. Есть возможность сохранения
def show_graph(region, pred_hour):
    global first_plot_url
    ow.clear_output()
    tup = (str(region), str(pred_hour))
    with ow:
        t = df.index[int(pred_hour):]
        graph1 = plotly.graph_objs.Scatter(x=t, 
                        y=df[str(region)].iloc[int(pred_hour):],
                        name = 'true',
                        mode = 'lines',
                        opacity = 1
                       )
        graph2 = plotly.graph_objs.Scatter(x=t,
                           y=region_data.get_group(tup).y,
                           name = 'forecast',
                           mode = 'lines',
                           opacity = 1)

        layout = dict(title=u"Предсказание для региона " + str(region))
        plotly.offline.iplot({'data': [graph1, graph2], 'layout': layout})
        first_plot_url = plotly.plotly.plot([graph1, graph2], filename='region sample', auto_open=False,)

In [102]:
# объект для отображения интерактивных графиков, он формируется в функции setlayout
ow = widgets.Output()

In [103]:
# Создаем интерфейс, контроллеры
# region_selector_map = widgets.Dropdown(options=regions, value=1075, description=u"Регион")
# region_selector_map.observe(region_selected, names="value")

# настройка слайдеров по дням
time_june = pd.date_range('2016-05-31 00:00:00', '2016-06-30 17:00:00', freq='D')
date_selector_map = widgets.SelectionSlider(
                                        description=u'Дата',
                                        options=[(item.strftime('%Y-%m-%d'),item) for item in time_june],
                                        continuous_update=False
                                        )

# настройка слайдеров по часам
date_selector_map.observe(setLayout, names="value")
hour_selector_map = widgets.SelectionSlider(description=u"Час",
                                       options=[(item, item) for item in range(24)],
                                       continuous_update=False,
                                       value=23)
hour_selector_map.observe(setLayout, names="value")

# настройка слайдеров для предсказаний
prediction_hour_slider = widgets.SelectionSlider(description=u"Предсказание",
                                                 options=[(item, item) for item in range(1,7)],
                                                 continuous_update=False,
                                                 value=1)
prediction_hour_slider.observe(setLayout, names="value")
prediction_hour_slider.observe(on_pred_hour_changed, names="value")

In [104]:
# Создаем карты и интерфейс
ny_map_real = Map(center=((ny_south + ny_north)/2, (ny_east + ny_west)/2),
                zoom=11)
ny_map_pred = Map(center=((ny_south + ny_north)/2, (ny_east + ny_west)/2),
                zoom=11)

# соединяем две карты вместе
map_center_link = link((ny_map_real, 'center'), (ny_map_pred, 'center'))
map_zoom_link = link((ny_map_real, 'zoom'), (ny_map_pred, 'zoom'))

box_layout = widgets.Layout(display='flex',
                            flex_flow='column',
                            width='50%')

# виджеты для выбранных регионов
label_region_clicked = widgets.Text(width='100%', disabled=True)
label_current = widgets.Text(width='100%', disabled=True)
label_true = widgets.Text(width='100%', disabled=True)
label_pred = widgets.Text(width='100%', disabled=True)
layout_true_pred = widgets.HBox(children=[label_region_clicked, label_current, label_true, label_pred])

# map_label_real = widgets.VBox([widgets.Text(u'Реальное число поездок', disabled=True), ny_map_real], layout=box_layout)
# map_label_pred = widgets.VBox([widgets.Text(u'Предсказанное число поездок', disabled=True), ny_map_pred], layout=box_layout)

maps_widget = widgets.HBox(children=[ny_map_real, ny_map_pred])
setLayout(0)

controllers = widgets.VBox(children=[date_selector_map, hour_selector_map, prediction_hour_slider])

In [105]:
widgets.VBox(children=[layout_true_pred, controllers, maps_widget])

Далее приводится два графика: на одном отображается фактическое число поездок в выбранном на карте, которая представлена ниже, регионе, на другом - предсказанное регрессором, который делает предсказание на выбранное число часов вперед. Соответсвенно на этом рисунке мы можем посмотреть на сколько сильно ошибается регрессор.

In [106]:
ow = widgets.Output()
display(ow)

Output()

На следующем интерактивном рисунке можно выбирать регион, дату и час дня, при этом для выбранных параметров будет отображаться предсказанные на 6 часов вперед и фактические значения числа поездок.

In [107]:
def change_region(change, date, hour):
    global second_plot_url
    
    print(date.to_pydatetime().replace(hour=hour))
    if date.to_pydatetime() == datetime(year=2016, month=5, day=31) and hour != 23:
        hour_selector.set_trait('value', 23)
        return
    if date.to_pydatetime() == datetime(year=2016, month=6, day=30) and hour > 17:
        hour_selector.set_trait('value', 17)
        return
    t = pd.date_range(date.replace(hour=hour), periods=7, freq='H')[1:]
    pred = hourly_data.get_group((str(change), date.strftime('%Y-%m-%d'), str(hour)))

    layout = dict(title=u"Предсказание на 6 часов вперед")

    graph1 = plotly.graph_objs.Scatter(x=t, 
                        y=df[str(change)].loc[t],
                        name = 'true',
                        mode = 'lines+markers'
                       )

    graph2 = plotly.graph_objs.Scatter(x=t,
                        y=pred.y,
                        name = 'forecast',
                        mode = 'lines+markers'
                       )

    plotly.offline.iplot({'data': [graph1, graph2], 'layout': layout})
    second_plot_url = plotly.plotly.plot([graph1, graph2], filename='predict 6 next hours', auto_open=False,)
    print (second_plot_url)

In [108]:
region_selector = widgets.Dropdown(options=regions, value=1075, description=u"Регион")
time_june = pd.date_range('2016-05-31 00:00:00', '2016-06-30 17:00:00', freq='D')
date_selector = widgets.SelectionSlider(
                                        description=u'Дата',
                                        options=[(item.strftime('%Y-%m-%d'),item) for item in time_june],
                                        continuous_update=False
                                        )
hour_selector = widgets.SelectionSlider(description=u"Час",
                                       options=[(item, item) for item in range(24)],
                                       continuous_update=False)
interact(change_region, change=region_selector, date=date_selector, hour=hour_selector);

interactive(children=(Dropdown(description='Регион', options=(1075, 1076, 1077, 1125, 1126, 1127, 1128, 1129, …